Import libraries

In [11]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import  precision_score, recall_score, f1_score, average_precision_score, precision_recall_curve, roc_auc_score,accuracy_score
import joblib
from sklearn.model_selection import KFold
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from keras import layers
import keras


Training dataset in one class neural network model

In [16]:
# load the csv
training_dataset = pd.read_csv('Dataset\Training_heart.csv')

X = training_dataset.drop(' HEALTHY HEART', axis=1)
y = training_dataset.drop([' PULSE', ' SpO2'], axis=1)

smote = SMOTE()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True,random_state=42)

X_train, y_train = smote.fit_resample(X_train, y_train)

# partitioning data for training and testing

ocnn_model = tf.keras.Sequential([
    layers.Dense(150, activation='relu', input_shape=(2,)), 
    layers.Dense(100, activation='relu'),
    layers.Dropout(0.25), 
    layers.Dense(50, activation='relu'),
    layers.Dropout(0.25), 
    layers.Flatten(),
    layers.Dense(1, activation='softmax')
])

# Compile the model
ocnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with only positive class samples
ocnn_model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split = 0.16)

# Evaluate the model on the positive class test set
loss = ocnn_model.evaluate(X_train, y_train)

# Predict on new samples
y_pred = ocnn_model.predict(X_test)
y_pred = np.where(y_pred == -1, 0, 1)


# Compute evaluation metrics
precision = precision_score(y_test, y_pred)
positive_percentage = y_test.sum()/ len(y_test)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)
auprc = average_precision_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)


# Print the evaluation scores
'''
precision = precision_score(y_val, y_test_pred)
recall = recall_score(y_val, y_test_pred)'''

print ("Positive percentage:", positive_percentage)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)  
print("Accuracy ", accuracy)
print("ROC AUC", roc_auc)
print("AUPRC:", auprc)

ocnn_model.save('heart_attack_monitor_OCNN.h5')

Epoch 1/20
793/793 [==============================] - 2s 2ms/step - loss: 0.2784 - accuracy: 0.5953 - val_loss: 0.3317 - val_accuracy: 0.0000e+00
Epoch 2/20
793/793 [==============================] - 1s 2ms/step - loss: 0.2081 - accuracy: 0.5953 - val_loss: 0.3500 - val_accuracy: 0.0000e+00
Epoch 3/20
793/793 [==============================] - 1s 1ms/step - loss: 0.1992 - accuracy: 0.5953 - val_loss: 0.2063 - val_accuracy: 0.0000e+00
Epoch 4/20
793/793 [==============================] - 1s 1ms/step - loss: 0.1940 - accuracy: 0.5953 - val_loss: 0.3047 - val_accuracy: 0.0000e+00
Epoch 5/20
793/793 [==============================] - 1s 1ms/step - loss: 0.1911 - accuracy: 0.5953 - val_loss: 0.4002 - val_accuracy: 0.0000e+00
Epoch 6/20
793/793 [==============================] - 1s 1ms/step - loss: 0.1866 - accuracy: 0.5953 - val_loss: 0.1942 - val_accuracy: 0.0000e+00
Epoch 7/20
793/793 [==============================] - 1s 1ms/step - loss: 0.1855 - accuracy: 0.5953 - val_loss: 0.2234 - val

Testing the OCNN model performance 

In [2]:
# Load a model
ocnn = keras.models.load_model('heart_attack_monitor_OCNN.h5')

val_dataset = pd.read_csv('Validate_heart.csv')
x_val = val_dataset.drop('Normal heart', axis=1)

y_val = val_dataset['Normal heart']  
y_val_features = np.reshape(y_val, (-1, 1))

y_pred = ocnn.predict(x_val)
y_pred = np.where(y_pred == -1, 0, 1)

positive_percentage = (y_val_features).sum() / len(y_val_features)
precision, recall, thresholds = precision_recall_curve(y_val_features, y_pred)


# Compute evaluation metrics
f1 = f1_score(y_val_features, y_pred)

roc_auc = roc_auc_score(y_val_features, y_pred)

# Print the evaluation scores

outlier_count = (y_val_features == -1).sum()

print ("Positive percentage:", positive_percentage)
print("Outlier count:", outlier_count)
print("Precision:", precision)
print("Recall:", recall)
print("Threshold:", thresholds)
print("ROC AUC", roc_auc)
print("F1-score:", f1)  
#print("Accuracy ", scores.mean())

# Calculate AUPRC
auprc = average_precision_score(y_val_features, y_pred)
print("AUPRC:", auprc)

781/781 [==============================] - 1s 1ms/step
Positive percentage: 0.9599679743795037
Outlier count: 500
Precision: [0.97998399 1.        ]
Recall: [1. 0.]
Threshold: [1]
ROC AUC 0.5
F1-score: 0.9898908208653457
AUPRC: 0.9799839871897518
